# Calculate Basic Statistics of Detailed Reviews

In [1]:
import gzip
import json
import re
import os
import sys
import numpy as np
import pandas as pd
pd.options.display.float_format = '{:,}'.format

**Specify your directory here:**

In [2]:
DIR = './'
DIR_GENRE = './genre/'

**Calculate statistics of book reviews (without spoiler tags)**

In [3]:
def count_reviews(file_name):
    print('counting file:', file_name)
    n_review = 0
    book_set, user_set = set(), set()
    print('current line: ', end='')
    with gzip.open(file_name) as fin:
        for l in fin:
            d = json.loads(l)
            if n_review % 1000000 == 0:
                print(n_review, end=',')
            n_review += 1
            book_set.add(d['book_id'])
            user_set.add(d['user_id'])
    print('complete')
    print('done!')
    return n_review, len(book_set), len(user_set)

In [4]:
n_review, n_book, n_user = count_reviews(os.path.join(DIR, 'goodreads_reviews_dedup.json.gz'))
df_stats_review = pd.DataFrame([n_review, n_book, n_user], dtype=float,
                               columns=['count'], index=['# review', '# book', '# user'])
display(df_stats_review)

counting file: ./goodreads_reviews_dedup.json.gz
current line: 0,1000000,2000000,3000000,4000000,5000000,6000000,7000000,8000000,9000000,10000000,11000000,12000000,13000000,14000000,15000000,complete
done!


,count
# review,"15,739,967.0"
# book,"2,080,190.0"
# user,"465,323.0"


**Calculate statistics of the review subset with spoiler tags**

In [5]:
def count_spoilers(file_name):
    print('counting file:', file_name)
    n_review, n_sentence, n_spoiler_review, n_spoiler_sentence = 0, 0, 0, 0
    book_set, user_set = set(), set()
    print('current line: ', end='')
    with gzip.open(file_name) as fin:
        for l in fin:
            d = json.loads(l)
            if n_review % 1000000 == 0:
                print(n_review, end=',')
            n_review += 1
            for _t, _ in d['review_sentences']:
                n_sentence += 1
                n_spoiler_sentence += _t
            n_spoiler_review += int(d['has_spoiler'])
            book_set.add(d['book_id'])
            user_set.add(d['user_id'])
    print('complete')
    print('done!')
    return n_review, n_sentence, n_spoiler_review, n_spoiler_sentence, len(book_set), len(user_set)

In [6]:
res = count_spoilers(os.path.join(DIR, 'goodreads_reviews_spoiler.json.gz'))
df_stats_spoiler = pd.DataFrame(res, columns=['count'], dtype=float, 
                               index=['# review', '# sentence', '# spoiler review', '# spoiler sentence',
                                      '# book', '# user'])
display(df_stats_spoiler)

counting file: ./goodreads_reviews_spoiler.json.gz
current line: 0,1000000,complete
done!


,count
# review,"1,378,033.0"
# sentence,"17,672,655.0"
# spoiler review,"89,627.0"
# spoiler sentence,"569,724.0"
# book,"25,475.0"
# user,"18,892.0"


**Calculate basic statistics of review files for each genre:**

In [7]:
genre_list = ['children', 'comics_graphic', 'fantasy_paranormal', 'history_biography',
              'mystery_thriller_crime', 'poetry', 'romance', 'young_adult']

def count_all_genres(genre_list):
    res = []
    for g in genre_list:
        n_review, n_book, n_user = count_reviews(
            os.path.join(DIR_GENRE, 'goodreads_reviews_'+g+'.json.gz'))
        res.append([n_review, n_book, n_user])
    df_stats_by_genre = pd.DataFrame(res, dtype = float, 
                                     columns = ['# review', '# book', '# user'],
                                     index = genre_list)
    return df_stats_by_genre

In [8]:
df_stats_by_genre = count_all_genres(genre_list)
display(df_stats_by_genre)

counting file: ./genre/goodreads_reviews_children.json.gz
current line: 0,complete
done!
counting file: ./genre/goodreads_reviews_comics_graphic.json.gz
current line: 0,complete
done!
counting file: ./genre/goodreads_reviews_fantasy_paranormal.json.gz
current line: 0,1000000,2000000,3000000,complete
done!
counting file: ./genre/goodreads_reviews_history_biography.json.gz
current line: 0,1000000,2000000,complete
done!
counting file: ./genre/goodreads_reviews_mystery_thriller_crime.json.gz
current line: 0,1000000,complete
done!
counting file: ./genre/goodreads_reviews_poetry.json.gz
current line: 0,complete
done!
counting file: ./genre/goodreads_reviews_romance.json.gz
current line: 0,1000000,2000000,3000000,complete
done!
counting file: ./genre/goodreads_reviews_young_adult.json.gz
current line: 0,1000000,2000000,complete
done!


,# review,# book,# user
children,"734,640.0","123,946.0","92,667.0"
comics_graphic,"542,338.0","89,311.0","59,347.0"
fantasy_paranormal,"3,424,641.0","258,212.0","256,088.0"
history_biography,"2,066,193.0","302,346.0","238,450.0"
mystery_thriller_crime,"1,849,236.0","218,987.0","203,655.0"
poetry,"154,555.0","36,412.0","47,400.0"
romance,"3,565,378.0","334,957.0","198,141.0"
young_adult,"2,389,900.0","93,267.0","209,152.0"
